# Debugger and Profiler

In [2]:
# install dependencies
!pip install smdebug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 2.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 1.1 MB/s eta 0:00:00:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [3]:
hyperparameters = {
    "batch_size": 2048,
    "gpu": True,
    "epoch": 2,
    "model": "resnet50",
}

In [12]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
                         

rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),

]

    
                                                         
         #TODO: Can you add the rules you want to track
         


In [14]:
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

#TODO: Can you create the profiler and debugger configs
profiler_config = ProfilerConfig(system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10))
debugger_config = DebuggerHookConfig(hook_parameters={"train.save_interval":"100", "test.save_interval":"10"})

In [38]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(role= sagemaker.get_execution_role(),
                    instance_count=1,
                    instance_type="ml.p3.2xlarge",
                    source_dir="scripts",
                    entry_point="pytorch_cifar_profiling.py",
                    hyperparameters= hyperparameters,
                    framework_version="1.8",
                    py_version="py36",
                    rules=rules,
                    profiler_config= profiler_config,
                    debugger_hook_config=debugger_config)
                    
                    #TODO: Create the estimator to train your model

In [39]:
estimator.fit(wait=True)

2022-08-21 15:34:59 Starting - Starting the training job...LossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
...
2022-08-21 15:35:50 Starting - Preparing the instances for training......
2022-08-21 15:36:50 Downloading - Downloading input data...
2022-08-21 15:37:30 Training - Downloading the training image........................
2022-08-21 15:41:32 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-21 15:41:32,763 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-21 15:41:32,784 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-21 15:41:32,794 sagemaker_pytorch_container.training INFO     Invoking user traini

In [44]:
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

Training jobname: pytorch-training-2022-08-21-15-34-58-970
Region: us-east-1


In [50]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

MissingCollectionFiles: Training job has ended. All the collection files could not be loaded

In [42]:
print(trial.tensor_names())
# TODO: Can you print the names of all the tensors that were tracked
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode

NameError: name 'trial' is not defined

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

In [ ]:
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [ ]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")